In [19]:
import os
import numpy as np
import pandas as pd

from scipy.io import wavfile
import scipy.fftpack as fftpack
import scipy.signal.windows as windows

import time

In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.multiclass import OneVsRestClassifier

from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import confusion_matrix, accuracy_score

In [3]:
DATASET_PATH = '.\data_v_7_stc'
FS = 16000

In [4]:
TRAIN_DIR = os.path.join(DATASET_PATH, 'audio')
train_file = [f for f in os.listdir(TRAIN_DIR) if os.path.isfile(os.path.join(TRAIN_DIR, f))]
train_wave = [w for w in train_file if w[-4:] == '.wav']

In [5]:
df = pd.read_csv(os.path.join(DATASET_PATH, 'meta/meta.txt'), sep='\t', names=['File', '_', '__', 'Duration', 'Class'])
df.drop(['_', '__'], axis=1, inplace=True)

In [6]:
TEST_DIR = os.path.join(DATASET_PATH, 'test')
test_file = [f for f in os.listdir(TEST_DIR) if os.path.isfile(os.path.join(TEST_DIR, f))]
test_wave = [w for w in test_file if w[-4:] == '.wav']

test_tag_split = [(f.split('_'))[:-1] for f in test_wave]
test_tag_list = [ [tag for tag in tag_split if tag != 't'] for tag_split in test_tag_split]
test_tag = ['_'.join(tag_list) for tag_list in test_tag_list]

In [7]:
df_test = pd.DataFrame(np.concatenate((test_wave, test_tag)).reshape(-1, 2, order='F'), columns=['File', 'Class'])

In [8]:
tags = np.unique(test_tag)
d_class = dict(zip(tags, range(tags.shape[0])))
d_class_inv = {val:key for key, val in d_class.items()}

In [9]:
def preproc_wavfile(full_file_name):
    fs, data = wavfile.read(full_file_name)
    data_scaled = signal_scaler(data)
    signal_part = signal_detector(data_scaled, fs//10, fs//10, 10)
    
    return data_scaled, fs, signal_part

def signal_scaler(data, maximum=2**15 - 1):
    scale = maximum / max(abs(data))
    
    return np.array(data*scale, dtype=data.dtype)

def std_average(data, window):
    N = data.shape[0]
    W = int(window)
    HW = int(window // 2)
    if (N <= W):
        return np.array([data.mean()], dtype=data.dtype)
    
    data_std  = np.zeros(int((2*(N//W + 1)),), dtype=data.dtype)
    
    data_std[0] = data[:HW].std()
    for i in range(data_std.shape[0] - 2):
        data_std[i+1] = data[i*HW:(i+2)*HW].std()
    data_std[-1] = data[-HW:].std()

    return data_std

def signal_detector(data, window, min_duration_frame, scale=10):
    data_std = std_average(data, window)
    
    std_filter = np.zeros(data_std.shape, dtype='bool')
    std_filter[data_std > max(data_std)/scale] = True
       
    while (all(std_filter)):
        scale /= 2
        std_filter = np.zeros(data_std.shape, dtype='bool')
        std_filter[data_std > max(data_std)/scale] = True
        
    if (not any(std_filter)):
        return []
    
    HW = window//2
    signal_frame = []
    front = np.argwhere(std_filter).reshape(-1,)[0]
    back = 0
    is_signal = std_filter[0]
    signal_duration = 0
    silence_duration = 0
    for i, flag in enumerate(std_filter):
        if (flag and not is_signal):
            silence_duration = (i - back + 1)*HW
            is_signal = True
            if (silence_duration > min_duration_frame):
                if (signal_duration >= min_duration_frame):
                    signal_frame.append((max(0,             (front-1)*HW),
                                         min(data.shape[0], (back   )*HW)))
                front = i     
        if (not flag and is_signal):
            back = i
            is_signal = False
            signal_duration = (back - front + 1)*HW
            
    if (std_filter[-1]):
        signal_frame.append((max(0,(front-1)*HW), data.shape[0]))
    else:
        signal_frame.append((max(0,             (front-1)*HW),
                             min(data.shape[0], (back   )*HW)))     
    
    return signal_frame

In [10]:
def process_oscilogram_feature(DIR, file_name_list):
    signal_parts = [None] * len(file_name_list)
    feat_ndarray = np.zeros((len(file_name_list), 4))
    for i, file_name in enumerate(file_name_list):
        data_scaled, _, signal_part = preproc_wavfile(os.path.join(DIR, file_name))
        feat_ndarray[i] = oscilogram_feature(data_scaled, signal_part)
        signal_parts[i] = signal_part
        
    return feat_ndarray, ['std_median', 'power_median', 'ratio', 'part_count'], signal_parts

def oscilogram_feature(data_scaled, signal_part):
    if (len(signal_part) == 0):
        return np.std(data_scaled), np.median(data_scaled**2), 1, 0
    
    N = data_scaled.shape[0]
    signal_data = np.ndarray((0,))
    signal_duration = 0
    for front, back in signal_part:
        signal_data = np.concatenate((signal_data, data_scaled[front:back]))
        signal_duration += back - front
    std_median = np.std(signal_data)
    power_median = np.median(signal_data**2)
    ratio = signal_duration / N
    part_count = len(signal_part)
    
    return std_median, power_median, ratio, part_count

In [11]:
train_oscilo_feat, _, _ = process_oscilogram_feature(TRAIN_DIR, df['File'])

In [12]:
test_oscilo_feat, _, _ = process_oscilogram_feature(TEST_DIR, df_test['File'])

In [13]:
def moving_average(data, window):
    s = pd.Series(data).rolling(window=window).mean()
    s = s.shift(-window//2).dropna().tolist()
    
    return s[:1]*(window//2) + s + s[-1:]*(window//2 - 1)

def exponential_smoothing(data, alpha):
    result = [data[0]] # first value is same as series
    for n in range(1, len(data)):
        result.append(alpha * data[n] + (1 - alpha) * result[n-1])
        
    return result

def fft(data, fs, func_avr=None, *param):
    data = np.array(data)
    if (data.shape[0] % fs != 0):
        data = np.concatenate((data, np.zeros((fs - data.shape[0] % fs, ) , dtype=data.dtype)), axis=0)
    
    N = data.shape[0]
    fn = fs//2  
    n = N//2
    
    X = fftpack.fft(data) / (N / 2.0)
    mag = np.abs(X / abs(X).max())[:n]
    X_db = 20*np.log10(mag)
    freq = fftpack.fftfreq(N, 1/fs)[:n]

    spec = X_db.reshape(fn, -1).mean(axis=1)
    if (func_avr is not None):
        spec = np.array(func_avr(spec, *param))
    
    return (spec, list(range(1, fn + 1)))

In [14]:
def process_spectral_feature(DIR, file_name_list, sample_rate=FS):
    signal_parts = [None] * len(file_name_list)
    feat_ndarray = np.zeros((len(file_name_list), sample_rate//2))
    for i, file_name in enumerate(file_name_list):
        data_scaled, fs, signal_part = preproc_wavfile(os.path.join(DIR, file_name))
        feat, freq = spectrum_feature(data_scaled, fs, signal_part)
        feat_ndarray[i], frequency_range = feat[:sample_rate//2], freq[:sample_rate//2]
        signal_parts[i] = signal_part
        
    return feat_ndarray, frequency_range, signal_parts

def spectrum_feature(data, fs, signal_part):
    if (len(signal_part) == 0):
        window = windows.hann(data.shape[0])
        return fft(window*data, fs, moving_average, fs//100)

    N = data.shape[0]
    signal_data = np.ndarray((0,))
    for front, back in signal_part:
        Xn = data[front:back]
        Xn_1 = np.roll(Xn, 1)
        Xn_1[0] = Xn_1[1]
        window = windows.hann(back - front)
        signal_data = np.concatenate((signal_data, window*(Xn - 0.95*Xn_1)))
        
    return fft(signal_data, fs, moving_average, fs//100)

In [15]:
train_spectr_feat, _, _ = process_spectral_feature(TRAIN_DIR, df['File'])

In [16]:
test_spectr_feat, _, _ = process_spectral_feature(TEST_DIR, df_test['File'])

In [17]:
def get_data(mode_list):
    X_train = np.ndarray((df.shape[0], 0))
    X_test = np.ndarray((df_test.shape[0], 0))
        
    for mode in mode_list:
        if (mode == 'oscilo'):
            X_train = np.concatenate((X_train, train_oscilo_feat), axis=1)
            X_test = np.concatenate((X_test, test_oscilo_feat), axis=1)
        elif (mode == 'spec'):
            X_train = np.concatenate((X_train, train_spectr_feat), axis=1)
            X_test = np.concatenate((X_test, test_spectr_feat), axis=1)
        
    y_train = (df['Class'].map(d_class)).values
    y_test = (df_test['Class'].map(d_class)).values
    
    return X_train, y_train, X_test, y_test

Все исследования проводятся на пятикратной кросс-валидации тренировочных данных.

Подбор гипперпараметров для классификаторов осуществлен в RnD.ipynb

Далее идет подбор гипперпараметров (alpha, beta, gamma) для ансамбля классификаторов (занимает порядка 5 часов).

В завершении подбирается порог отсечения класса "unknown" для открытой задачи классификации (open_threshold). Он рассчитывается как медианное значение вероятности распознавания неверного класса на всей кросс-валидации.

---

Так как процесс подбора гипперпараметров для ансамбля классификаторов занимает много времени, нижеследующей ячейкой можно задать заранее мной полученные для них значения и порог отсечения класса "unknown" для открытой задачи классификации.

In [24]:
mode_list = ['oscilo', 'spec']

hyper_params = {}
param_grid = np.linspace(0.0, 1.0, 11)
for key, index in zip(('alpha', 'beta', 'gamma'), (3, 5, 7)):
    hyper_params[key] = param_grid[index]
    
open_threshold = np.array([0.5091])

---

Если была запущена предыдущая ячейка, то 3 нижеследующих ячейки можно пропустить.

In [ ]:
n_splits = 5
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=17)

mode_list = ['oscilo', 'spec']
X, y, _, _ = get_data(mode_list)

cls_list = ['logit', 'svc', 'rfc', 'gbc']
train_proba = {}
valid_proba = {}
for cls in cls_list:
    train_proba[cls] = n_splits * [None]
    valid_proba[cls] = n_splits * [None]

print(mode_list)
print()
for i, (train_idx, valid_idx) in enumerate(skf.split(X, y)):
    print('Path #', i+1)
    tic = time.time()
    X_train, y_train = X[train_idx], y[train_idx]
    X_valid, y_valid = X[valid_idx], y[valid_idx]

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_valid_scaled = scaler.transform(X_valid)

    tic_tic = time.time()
    logit = LogisticRegression(C=5, penalty='l2', solver='lbfgs', multi_class='ovr', n_jobs=-1, random_state=17)
    logit.fit(X_train_scaled, y_train)
    train_proba['logit'][i] = logit.predict_proba(X_train_scaled)
    valid_proba['logit'][i] = logit.predict_proba(X_valid_scaled)
    toc_toc = time.time()
    print('Logistic Regression: train acc = %.3f, valid acc = %.3f (%s)' %
          (accuracy_score(y_train, [np.argmax(probas) for probas in train_proba['logit'][i]]),
           accuracy_score(y_valid, [np.argmax(probas) for probas in valid_proba['logit'][i]]),
           time.strftime('%Hh %Mmin %Ss', time.gmtime(toc_toc - tic_tic))))
    
    tic_tic = time.time()
    svc = SVC(C=10, kernel='rbf', gamma='auto', decision_function_shape='ovr', random_state=17, probability=True)
    svc.fit(X_train_scaled, y_train)
    train_proba['svc'][i] = svc.predict_proba(X_train_scaled)
    valid_proba['svc'][i] = svc.predict_proba(X_valid_scaled)
    toc_toc = time.time()
    print('Support Vector Classifier: train acc = %.3f, valid acc = %.3f (%s)' %
          (accuracy_score(y_train, [np.argmax(probas) for probas in train_proba['svc'][i]]),
           accuracy_score(y_valid, [np.argmax(probas) for probas in valid_proba['svc'][i]]),
           time.strftime('%Hh %Mmin %Ss', time.gmtime(toc_toc - tic_tic))))

    tic_tic = time.time()
    rfc = OneVsRestClassifier(RandomForestClassifier(n_estimators=250, random_state=17, n_jobs=-1), n_jobs=-1)
    rfc.fit(X_train, y_train)
    train_proba['rfc'][i] = rfc.predict_proba(X_train)
    valid_proba['rfc'][i] = rfc.predict_proba(X_valid)
    toc_toc = time.time()
    print('Random Forest Classifier: train acc = %.3f, valid acc = %.3f (%s)' %
          (accuracy_score(y_train, [np.argmax(probas) for probas in train_proba['rfc'][i]]),
           accuracy_score(y_valid, [np.argmax(probas) for probas in valid_proba['rfc'][i]]),
           time.strftime('%Hh %Mmin %Ss', time.gmtime(toc_toc - tic_tic))))
    
    tic_tic = time.time()
    gbc = OneVsRestClassifier(GradientBoostingClassifier(n_estimators=150, max_depth=3, random_state=17), n_jobs=-1)
    gbc.fit(X_train, y_train)
    train_proba['gbc'][i] = gbc.predict_proba(X_train)
    valid_proba['gbc'][i] = gbc.predict_proba(X_valid)
    toc_toc = time.time()
    print('Gradient Boosting Classifier: train acc = %.3f, valid acc = %.3f (%s)' %
          (accuracy_score(y_train, [np.argmax(probas) for probas in train_proba['gbc'][i]]),
           accuracy_score(y_valid, [np.argmax(probas) for probas in valid_proba['gbc'][i]]),
           time.strftime('%Hh %Mmin %Ss', time.gmtime(toc_toc - tic_tic))))
    
    toc = time.time()
    print('Wall time: %s' % time.strftime('%Hh %Mmin %Ss', time.gmtime(toc - tic)))
    print()

['oscilo', 'spec']

Path # 1
Logistic Regression: train acc = 0.922, valid acc = 0.912 (00h 06min 50s)
Support Vector Classifier: train acc = 0.987, valid acc = 0.973 (00h 16min 19s)
Random Forest Classifier: train acc = 1.000, valid acc = 0.967 (00h 08min 15s)
Gradient Boosting Classifier: train acc = 0.998, valid acc = 0.967 (01h 41min 26s)
Wall time: 02h 12min 54s

Path # 2
Logistic Regression: train acc = 0.928, valid acc = 0.914 (00h 06min 05s)
Support Vector Classifier: train acc = 0.988, valid acc = 0.968 (00h 16min 03s)
Random Forest Classifier: train acc = 1.000, valid acc = 0.966 (00h 08min 18s)


In [ ]:
param_grid = np.linspace(0.0, 1.0, 11)
valid_accuracy = np.zeros(tuple([n_splits] + [11]*(len(cls_list) - 1)))

for i, (train_idx, valid_idx) in enumerate(skf.split(X, y)):
    X_train, y_train = X[train_idx], y[train_idx]
    X_valid, y_valid = X[valid_idx], y[valid_idx]
    
    for j, alpha in enumerate(param_grid):
        for k, beta in enumerate(param_grid):
            for m, gamma in enumerate(param_grid):
                ensemble_proba = gamma*(beta*(alpha*valid_proba['logit'][i] \
                                              + (1.0 - alpha)*valid_proba['rfc'][i]) \
                                        + (1.0 - beta)*valid_proba['gbc'][i]) \
                                 + (1.0 - gamma)*valid_proba['svc'][i]
                valid_accuracy[i][j][k][m] = \
                    accuracy_score(y_valid, [np.argmax(probas) for probas in ensemble_proba])

#'logit', 'rfc', 'gbc', 'svc'
mean_valid_accuracy = np.mean(valid_accuracy, axis=0)
arg_max_ravel = np.argmax(mean_valid_accuracy)
idx = np.unravel_index(arg_max_ravel, tuple([11]*(len(cls_list) - 2)))
print(mean_valid_accuracy[idx], idx)

In [ ]:
hyper_params = {}
for key, index in zip(('alpha', 'beta', 'gamma'), idx):
    hyper_params[key] = param_grid[index]

false_proba_predict = []
for i, (train_idx, valid_idx) in enumerate(skf.split(X, y)):
    X_train, y_train = X[train_idx], y[train_idx]
    X_valid, y_valid = X[valid_idx], y[valid_idx]
    
    ensemble_proba = hyper_params['gamma']*(hyper_params['beta']*(hyper_params['alpha']*valid_proba['logit'][i] \
                                                                  + (1.0 - hyper_params['alpha'])*valid_proba['rfc'][i]) \
                                            + (1.0 - hyper_params['beta'])*valid_proba['gbc'][i]) \
                     + (1.0 - hyper_params['gamma'])*valid_proba['svc'][i]

    ensemble_proba = np.array(ensemble_proba)
    ensemble_class = np.array([np.argmax(probas) for probas in ensemble_proba])
    false_predict_proba += np.array([ensemble_proba[i, cls] for i, cls in enumerate(ensemble_class)]) \
                           [y_valid != ensemble_class].tolist()

open_threshold = np.median(false_predict_proba)
print('threshold = %.4f' % open_threshold)

In [ ]:
X_train, y_train, X_test, y_test = get_data(mode_list)

test_close_index = (df_test['Class'] != 'unknown').values

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

logit = LogisticRegression(C=5, penalty='l2', solver='lbfgs', multi_class='ovr', n_jobs=-1, random_state=17)
logit.fit(X_train_scaled, y_train)
logit_proba = logit.predict_proba(X_test_scaled)

svc = SVC(C=10, kernel='rbf', gamma='auto', decision_function_shape='ovr', random_state=17, probability=True)
svc.fit(X_train_scaled, y_train)
svc_proba = svc.predict_proba(X_test_scaled)

rfc = OneVsRestClassifier(RandomForestClassifier(n_estimators=250, random_state=17, n_jobs=-1), n_jobs=-1)
rfc.fit(X_train, y_train)
rfc_proba = rfc.predict_proba(X_test)

gbc = OneVsRestClassifier(GradientBoostingClassifier(n_estimators=150, max_depth=3, random_state=17), n_jobs=-1)
gbc.fit(X_train, y_train)
gbc_proba = gbc.predict_proba(X_test)

ensemble_proba = hyper_params['gamma']*(hyper_params['beta']*(hyper_params['alpha']*logit_proba \
                                                              + (1.0 - hyper_params['alpha'])*rfc_proba) \
                                        + (1.0 - hyper_params['beta'])*gbc_proba) \
                 + (1.0 - hyper_params['gamma'])*svc_proba
        
ensemble_class_close = np.array([np.argmax(probas) for probas in ensemble_proba])

print(accuracy_score(y_test[test_close_index], ensemble_class_close[test_close_index]))
cfm_close = confusion_matrix(ensemble_class_close[test_close_index], y_test[test_close_index])
pd.DataFrame(np.vstack((cfm_close, cfm_close.sum(axis=0),
                        [round(cfm_close[i, i] / cfm_close.sum(axis=0)[i], ndigits=2) for i in range(cfm_close.shape[1])])),
             index=list(d_class.keys())[:-1] + ['Total', 'Recall'],
             columns=[str[:4] for str in list(d_class.keys())[:-1]])

In [ ]:
df_result_close = pd.concat([df_test['File'],
                            pd.Series([np.max(proba) for proba in ensemble_proba], name='Score'),
                            pd.Series([d_class_inv[cls] for cls in ensemble_class_close], name='Class')], axis=1)
df_result_close.to_csv('./result_close.txt', sep='\t', header=False, index=False, float_format='%.3f')

In [ ]:
ensemble_class_open = ensemble_class_close.copy()
ensemble_class_open[[np.max(proba) for proba in ensemble_proba] < open_threshold] = d_class['unknown']

print(accuracy_score(y_test, ensemble_class_open))
cfm_open = confusion_matrix(ensemble_class_open, y_test)
pd.DataFrame(np.vstack((cfm_open, cfm_open.sum(axis=0),
                        [round(cfm_open[i, i] / cfm_open.sum(axis=0)[i], ndigits=2) for i in range(cfm_open.shape[0])])),
             index=list(d_class.keys()) + ['Total', 'Recall'],
             columns=[str[:4] for str in list(d_class.keys())])

In [ ]:
df_result_open = pd.concat([df_test['File'],
                            pd.Series([np.max(proba) for proba in ensemble_proba], name='Score'),
                            pd.Series([d_class_inv[cls] for cls in ensemble_class_open], name='Class')], axis=1)
df_result_open.to_csv('./result_open.txt', sep='\t', header=False, index=False, float_format='%.3f')